# Scenario 05: Declarative Agents and Workflows

**Estimated Time**: 30 minutes

## Learning Objectives
- Define agents using YAML configuration with **Microsoft Agent Framework**
- Configure workflows declaratively
- Load agents using `AgentFactoryLoader` (production) and legacy loaders
- Switch between imperative and declarative patterns

## Prerequisites
- Completed Scenario 04 (Deterministic Workflows)
- Basic understanding of YAML syntax
- Azure OpenAI credentials configured in `.env`

In [1]:
# Load environment and configure paths
import sys
from pathlib import Path

# Add project root to path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(project_root / ".env")

print(f"✅ Project root: {project_root}")

✅ Project root: C:\Users\jonasrotter\OneDrive - Microsoft\Desktop\Jonas Privat\MyCodingProjects\agents-workshop


In [ ]:
# Setup and imports
import os
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import Agent Framework classes (NEW - production approach)
from agent_framework import ChatAgent
from agent_framework_declarative import AgentFactory

# Import workshop loaders (supports both legacy and new formats)
from src.agents.declarative import (
    # NEW: Agent Framework loader
    AgentFactoryLoader,
    load_agent_from_yaml,
    load_agents_with_factory,
    # Legacy loaders (backward compatible)
    DeclarativeAgent,
    DeclarativeAgentLoader,
    DeclarativeWorkflowLoader,
    load_agents_from_config,
    load_workflows_from_config,
)
from src.common.yaml_loader import (
    YAMLLoader,
    AgentConfig,
    WorkflowConfig,
    validate_agent_yaml,
    validate_workflow_yaml,
)

print("✅ Imports successful")
print(f"✅ AgentFactory from: {AgentFactory.__module__}")

Patch applied successfully
✓ Applied clr_loader patch for .NET 10+ compatibility
[tfm] Applying fix for version: 10.0.1 -> net10.0
[floor_version] Applying fix for version: 10.0.1 -> 10.0.0


## Part 1: Agent Framework YAML Format (Production)

The **Microsoft Agent Framework** uses a standardized `kind: Prompt` YAML schema:

| Field | Type | Required | Description |
|-------|------|----------|-------------|
| `kind` | string | Yes | Agent type (e.g., `Prompt`) |
| `name` | string | Yes | Unique identifier |
| `model.id` | string | Yes | Model name (e.g., `gpt-4.1-mini`) |
| `model.provider` | string | Yes | Provider: `AzureOpenAI.Chat`, `OpenAI.Chat`, etc. |
| `model.connection` | object | No | Connection settings (endpoint) |
| `model.options` | object | No | Temperature, maxOutputTokens |
| `instructions` | string | Yes | System prompt |
| `tools` | list | No | Tool definitions |

**Important**: Provider must include the suffix (e.g., `AzureOpenAI.Chat` not just `AzureOpenAI`)

In [ ]:
# Agent Framework YAML format (production)
import tempfile
import yaml

# Get Azure endpoint from environment
azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT", "https://your-endpoint.openai.azure.com")

agent_framework_yaml = f"""
kind: Prompt
name: example_agent
model:
  id: gpt-4.1-mini
  provider: AzureOpenAI.Chat
  connection:
    kind: remote
    endpoint: {azure_endpoint}
  options:
    temperature: 0.7
    maxOutputTokens: 2048
instructions: |
  You are a helpful assistant.
  Be concise and accurate.
tools: []
"""

print("Agent Framework YAML format:")
print(agent_framework_yaml)

# Write to temp file and load with AgentFactory
with tempfile.NamedTemporaryFile(mode='w', suffix='.yaml', delete=False) as f:
    f.write(agent_framework_yaml)
    temp_path = f.name

try:
    factory = AgentFactory()
    agent = factory.create_agent_from_yaml_path(temp_path)
    print(f"\n✅ Created agent: {type(agent).__name__}")
    print(f"   Agent type: ChatAgent = {isinstance(agent, ChatAgent)}")
finally:
    os.unlink(temp_path)

In [ ]:
# Common mistakes with Agent Framework format
print("Common Provider Mistakes:")
print()

# Wrong: Missing .Chat suffix
wrong_provider = "AzureOpenAI"
correct_provider = "AzureOpenAI.Chat"
print(f"  ❌ Wrong:   provider: {wrong_provider}")
print(f"  ✅ Correct: provider: {correct_provider}")
print()

# List of valid providers
valid_providers = [
    "AzureOpenAI.Chat",
    "AzureOpenAI.Assistants", 
    "AzureOpenAI.Responses",
    "OpenAI.Chat",
    "OpenAI.Assistants",
    "OpenAI.Responses",
    "Anthropic.Chat",
]
print("Valid providers:")
for p in valid_providers:
    print(f"  - {p}")

## Part 2: Loading Agents with AgentFactoryLoader

The `AgentFactoryLoader` uses the official Agent Framework to create `ChatAgent` instances.

Directory structure:
```
configs/
  agents/
    research_agent.yaml      # kind: Prompt format
    summarizer_agent.yaml    # kind: Prompt format
```

In [ ]:
# View the actual YAML config (Agent Framework format)
config_path = project_root / "configs" / "agents" / "research_agent.yaml"

print(f"📄 {config_path.name}:")
print("-" * 50)
print(config_path.read_text())
print("-" * 50)

In [ ]:
# Load all agents using AgentFactoryLoader (NEW - production approach)
agent_loader = AgentFactoryLoader(
    agents_dir=project_root / "configs" / "agents"
)
agents = agent_loader.load_all()

print(f"✅ Loaded {len(agents)} agents using AgentFactory:")
for name in agent_loader.list_agents():
    agent = agent_loader.get_agent(name)
    print(f"  - {name}: {type(agent).__name__}")

In [ ]:
# Convenience function: load a single agent from YAML path
research_agent = load_agent_from_yaml(
    project_root / "configs" / "agents" / "research_agent.yaml"
)

print(f"✅ Loaded agent: {type(research_agent).__name__}")
print(f"   Is ChatAgent: {isinstance(research_agent, ChatAgent)}")

# Alternative: load all agents at once
all_agents = load_agents_with_factory(
    agents_dir=project_root / "configs" / "agents"
)
print(f"\n✅ All agents: {list(all_agents.keys())}")

## Part 3: Legacy Format (Backward Compatibility)

The workshop also supports a **legacy YAML format** for backward compatibility.
This uses `DeclarativeAgentLoader` instead of `AgentFactoryLoader`.

| Legacy Field | Agent Framework Equivalent |
|--------------|---------------------------|
| `model.provider: azure_openai` | `model.provider: AzureOpenAI.Chat` |
| `model.deployment: gpt-4.1-mini` | `model.id: gpt-4.1-mini` |
| `model.temperature: 0.7` | `model.options.temperature: 0.7` |
| `max_tokens: 4096` | `model.options.maxOutputTokens: 4096` |

**Recommendation**: Use Agent Framework format for new projects.

In [ ]:
# Legacy format example (for reference)
legacy_yaml = """
name: example_agent
model:
  provider: azure_openai
  deployment: gpt-4.1-mini
  temperature: 0.7
instructions: |
  You are a helpful assistant.
  Be concise and accurate.
tools:
  - search_web
max_tokens: 2048
"""

# Validate legacy YAML (uses Pydantic validation)
legacy_config = validate_agent_yaml(legacy_yaml)
print("Legacy format fields:")
print(f"  name: {legacy_config.name}")
print(f"  model.provider: {legacy_config.model.provider}")
print(f"  model.deployment: {legacy_config.model.deployment}")
print(f"  model.temperature: {legacy_config.model.temperature}")
print(f"  tools: {legacy_config.tools}")

# Create legacy DeclarativeAgent (mock - no real LLM)
legacy_agent = DeclarativeAgent(legacy_config)
print(f"\n✅ Legacy agent: {legacy_agent}")

## Part 4: Workflow Configuration Schema

Workflows are defined declaratively (uses custom WorkflowEngine - not part of agent-framework-declarative):

| Field | Type | Required | Description |
|-------|------|----------|-------------|
| `name` | string | Yes | Workflow identifier |
| `description` | string | No | Human-readable description |
| `steps` | list | Yes | Ordered workflow steps |
| `on_error` | object | No | Error handling strategy |

In [ ]:
# Example workflow configuration
workflow_yaml = """
name: simple_pipeline
description: A simple two-step pipeline

steps:
  - name: research
    agent: research_agent
    prompt: "Research this topic: {topic}"
    outputs:
      - findings

  - name: summarize
    agent: summarizer_agent
    prompt: "Summarize: {findings}"
    outputs:
      - summary

on_error:
  strategy: retry
"""

workflow_config = validate_workflow_yaml(workflow_yaml)
print(f"Workflow: {workflow_config.name}")
print(f"Steps: {[s.name for s in workflow_config.steps]}")
print(f"Error strategy: {workflow_config.on_error.strategy}")

In [ ]:
# Load workflow config from file using YAMLLoader
yaml_loader = YAMLLoader(base_path=project_root)
pipeline_config = yaml_loader.load_workflow(
    "configs/workflows/research_pipeline.yaml"
)

print(f"Pipeline: {pipeline_config.name}")
print(f"Description: {pipeline_config.description}")
print(f"\nSteps:")
for step in pipeline_config.steps:
    print(f"  {step.name}:")
    print(f"    agent: {step.agent}")
    print(f"    outputs: {step.outputs}")
    if step.retry:
        print(f"    retry: {step.retry.max_attempts} attempts")

## Part 5: Loading Complete Workflows with Agents

Workflows require agents to execute. We can combine `AgentFactoryLoader` (for agents)
with `DeclarativeWorkflowLoader` (for workflows) to build complete pipelines.

**Note**: The workflow loader uses a custom `WorkflowEngine` since `agent-framework-declarative` 
doesn't include workflow support.

In [ ]:
# Load agents using AgentFactoryLoader (Agent Framework format)
factory_loader = AgentFactoryLoader(
    agents_dir=project_root / "configs" / "agents"
)
chat_agents = factory_loader.load_all()

print(f"✅ Loaded {len(chat_agents)} agents with AgentFactory:")
for name in factory_loader.list_agents():
    agent = factory_loader.get_agent(name)
    print(f"  - {name}: {type(agent).__name__}")

# Note: Workflows use a separate workflow config format
# The workflow YAML references agent names, not the agent objects directly
print(f"\n📋 Workflow config references these agent names:")
print(f"   {list(chat_agents.keys())}")

In [ ]:
# Convenience functions for loading
# load_agents_with_factory() - loads agents using Agent Framework format
all_chat_agents = load_agents_with_factory(
    agents_dir=project_root / "configs" / "agents"
)
print(f"✅ Loaded agents (AgentFactory): {list(all_chat_agents.keys())}")

# View the workflow configuration
workflow_path = project_root / "configs" / "workflows" / "research_pipeline.yaml"
print(f"\n📄 Workflow config ({workflow_path.name}):")
print(workflow_path.read_text()[:500] + "...")

## Part 6: Runtime Behavior Modification

One of the key benefits of declarative configuration is modifying behavior
without code changes.

In [ ]:
# Create a modified agent using Agent Framework format
modified_agent_yaml = f"""
kind: Prompt
name: strict_research_agent
model:
  id: gpt-4o
  provider: AzureOpenAI.Chat
  connection:
    kind: remote
    endpoint: {azure_endpoint}
  options:
    temperature: 0.1
    maxOutputTokens: 3000
instructions: |
  You are a STRICT research assistant.
  
  RULES:
  - Only cite peer-reviewed sources
  - Include publication dates for all sources
  - Rate confidence level (high/medium/low) for each finding
  - Maximum 5 bullet points per source
tools: []
"""

# Write to temp file and create agent
with tempfile.NamedTemporaryFile(mode='w', suffix='.yaml', delete=False) as f:
    f.write(modified_agent_yaml)
    temp_path = f.name

try:
    modified_agent = AgentFactory().create_agent_from_yaml_path(temp_path)
    print(f"✅ Created modified agent: {type(modified_agent).__name__}")
finally:
    os.unlink(temp_path)

In [ ]:
# Compare Agent Framework configurations (YAML-based comparison)
# Note: ChatAgent objects don't expose temperature/max_tokens as attributes
# Instead, compare the YAML configurations directly

original_config = yaml.safe_load(agent_framework_yaml)
modified_config = yaml.safe_load(modified_agent_yaml)

print("Configuration Comparison (from YAML):")
print()
print(f"{'Field':<25} {'Original':<20} {'Modified':<20}")
print("-" * 65)

# Extract model options safely
original_opts = original_config.get('model', {}).get('options', {})
modified_opts = modified_config.get('model', {}).get('options', {})

print(f"{'Model ID':<25} {original_config['model']['id']:<20} {modified_config['model']['id']:<20}")
print(f"{'Temperature':<25} {original_opts.get('temperature', 'N/A'):<20} {modified_opts.get('temperature', 'N/A'):<20}")
print(f"{'Max Tokens':<25} {original_opts.get('maxOutputTokens', 'N/A'):<20} {modified_opts.get('maxOutputTokens', 'N/A'):<20}")
print()
print("💡 Note: ChatAgent encapsulates model settings internally.")
print("   To compare configurations, parse the YAML source files.")

## Part 7: Validation and Error Handling

Different formats use different validators:

- **Agent Framework format** (`kind: Prompt`): Use `AgentFactory` / `load_agent_from_yaml()`
- **Legacy format** (`model.provider: azure_openai`): Use `YAMLLoader.load_agent()` with Pydantic
- **Workflow format**: Use `YAMLLoader.load_workflow()` (custom format)

In [ ]:
# Various validation scenarios
test_cases = [
    # Missing required field
    ("""
name: test_agent
model:
  provider: azure_openai
  deployment: gpt-4
# Missing instructions!
""", "Missing 'instructions' field"),
    
    # Invalid name pattern
    ("""
name: 123_invalid
model:
  provider: azure_openai
  deployment: gpt-4
instructions: This is a test agent.
""", "Invalid name pattern (must start with letter)"),
    
    # Temperature out of range
    ("""
name: test_agent
model:
  provider: azure_openai
  deployment: gpt-4
  temperature: 5.0  # Max is 2.0
instructions: This is a test agent.
""", "Temperature out of range"),
]

for yaml_content, description in test_cases:
    try:
        validate_agent_yaml(yaml_content)
        print(f"❌ {description}: Should have failed!")
    except Exception as e:
        print(f"✅ {description}: Caught correctly")

## Part 8: Best Practices

### Configuration Organization

```
configs/
  agents/
    research_agent.yaml      # kind: Prompt (Agent Framework)
    summarizer_agent.yaml    # kind: Prompt (Agent Framework)
  workflows/
    research_pipeline.yaml   # Custom workflow format
  environments/
    dev.yaml                 # Development overrides
    prod.yaml                # Production settings
```

### YAML Format Selection

| Use Case | Format | Loader |
|----------|--------|--------|
| Production apps | `kind: Prompt` | `AgentFactoryLoader` |
| Workshop exercises | Legacy | `DeclarativeAgentLoader` |
| Custom workflows | Legacy | `DeclarativeWorkflowLoader` |

### Naming Conventions

- Use `snake_case` for names
- Be descriptive: `research_agent` not `ra`
- Include purpose in workflow names: `content_review_pipeline`

### Version Control

- Track all config changes in git
- Use meaningful commit messages for config changes
- Consider config file validation in CI/CD

In [ ]:
# Example: Config validation function for CI/CD
from pathlib import Path

def validate_all_configs(base_path: Path) -> list[str]:
    """Validate all configuration files.
    
    Uses AgentFactoryLoader for agents (Agent Framework format)
    and YAMLLoader for workflows (custom format).
    """
    errors = []
    
    # Validate agents using AgentFactoryLoader (Agent Framework format)
    agents_dir = base_path / "configs" / "agents"
    if agents_dir.exists():
        print("Validating agents (Agent Framework format):")
        for file in agents_dir.glob("*.yaml"):
            try:
                # Use AgentFactory to validate Agent Framework YAML
                agent = load_agent_from_yaml(file)
                print(f"  ✅ {file.name} -> {type(agent).__name__}")
            except Exception as e:
                errors.append(f"{file.name}: {e}")
                print(f"  ❌ {file.name}: {e}")
    
    # Validate workflows using YAMLLoader (custom workflow format)
    workflows_dir = base_path / "configs" / "workflows"
    if workflows_dir.exists():
        print("\nValidating workflows (custom format):")
        yaml_loader = YAMLLoader(base_path)
        for file in workflows_dir.glob("*.yaml"):
            try:
                yaml_loader.load_workflow(file)
                print(f"  ✅ {file.name}")
            except Exception as e:
                errors.append(f"{file.name}: {e}")
                print(f"  ❌ {file.name}: {e}")
    
    return errors

print("Config Validation:")
validation_errors = validate_all_configs(project_root)
print(f"\nTotal errors: {len(validation_errors)}")

## 🎯 Exercise: Create an Agent with Agent Framework Format

Create a new agent configuration for a "fact_checker_agent" using the **Agent Framework** format:

1. Use `kind: Prompt`
2. Use `model.provider: AzureOpenAI.Chat` with `model.id: gpt-4o`
3. Set low temperature (0.2) in `model.options`
4. Add instructions focused on verifying claims
5. Set `maxOutputTokens: 1500`

### Your Task:

1. Write the YAML configuration below
2. Save to a temp file
3. Load with `AgentFactory`

In [ ]:
# Your solution here - Agent Framework format
fact_checker_yaml = f"""
kind: Prompt
name: fact_checker_agent
model:
  id: gpt-4o
  provider: AzureOpenAI.Chat
  connection:
    kind: remote
    endpoint: {azure_endpoint}
  options:
    temperature: 0.2
    maxOutputTokens: 1500
instructions: |
  You are a fact-checking specialist.
  
  Your responsibilities:
  1. Verify claims against reliable sources
  2. Rate claim accuracy (True/False/Partially True/Unverifiable)
  3. Provide evidence for your assessment
  4. Note any important context or caveats
tools: []
"""

# Save and load with AgentFactory
with tempfile.NamedTemporaryFile(mode='w', suffix='.yaml', delete=False) as f:
    f.write(fact_checker_yaml)
    temp_path = f.name

try:
    fact_checker = AgentFactory().create_agent_from_yaml_path(temp_path)
    print(f"✅ Created fact_checker_agent: {type(fact_checker).__name__}")
    print(f"   Is ChatAgent: {isinstance(fact_checker, ChatAgent)}")
except Exception as e:
    print(f"❌ Failed: {e}")
finally:
    os.unlink(temp_path)

## Summary

In this scenario, you learned:

1. **Agent Framework Format**: Define agents with `kind: Prompt`, `model.provider: AzureOpenAI.Chat`
2. **AgentFactoryLoader**: Production loader creating `ChatAgent` instances
3. **Legacy Format**: Backward-compatible `model.provider: azure_openai` format
4. **Workflow Configuration**: Define multi-step workflows declaratively
5. **Format Selection**: When to use Agent Framework vs Legacy loaders
6. **Best Practices**: Organization, naming, version control

### Key Differences

| Feature | Agent Framework | Legacy |
|---------|-----------------|--------|
| Provider | `AzureOpenAI.Chat` | `azure_openai` |
| Model | `model.id` | `model.deployment` |
| Options | `model.options.temperature` | `model.temperature` |
| Loader | `AgentFactoryLoader` | `DeclarativeAgentLoader` |
| Returns | `ChatAgent` | `DeclarativeAgent` |

### Next Steps

- **Scenario 6**: Moderating Agent Discussions
- Explore environment-specific configuration overrides
- Build a configuration management UI